# Generate Piano Notes using Vital Preset

This notebook demonstrates how to load the `piano.vital` preset, apply necessary parameter fixes, and generate audio notes.

**NOTE:** The `piano.vital` preset relies on external assets (Samples and Custom Wavetables) which cannot be loaded via the current parameter mapping script. As a result, the intended piano sound (Sample) is missing, and the oscillator defaults to a raw Saw wave (Init). We apply parameter fixes to shape this Saw wave into a more usable sound and flush the buffer to prevent artifacts.

In [ ]:
import os
import sys
import numpy as np
import soundfile as sf
import mido
from pedalboard import load_plugin
import IPython.display as ipd

# Add project root to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

from src.data.vst.core import load_preset

# Paths
PLUGIN_PATH = "../../plugins/Vital.vst3"
PRESET_PATH = "piano.vital"
OUTPUT_DIR = "generated_notes"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Load Plugin
plugin = load_plugin(PLUGIN_PATH)
print("Plugin loaded.")

In [ ]:
# Load Preset
load_preset(plugin, PRESET_PATH)
print("Preset loaded.")

In [ ]:
# Apply Parameter Fixes
# NOTE: The 'piano.vital' preset relies on a Sample and a Custom Wavetable.
# The current loader cannot load these external assets, so they are skipped.
# As a result, the Sample is silent, and Oscillator 1 defaults to a raw Saw wave.
# We apply fixes to make this 'Init' Saw wave sound more like a pluck/piano using the filter.

def fix_piano_preset(plugin):
    # 1. Flush buffer to clear any initial noise
    # Vital is an instrument, so we must use the instrument process signature (no input audio)
    plugin.process([], 0.1, 44100, num_channels=2, reset=True)
    
    updates = {
        # Enable Oscillator 1 as fallback (since Sample is missing)
        "oscillator_1_level": 1.0,
        "oscillator_1_destination": 0.0, # Route to Filter 1
        
        # Shape the Saw wave with Filter 1
        "filter_1_on": 1.0,
        "filter_1_mix": 1.0,       # 100% Wet
        "filter_1_cutoff": 0.4,    # Lower cutoff (~60 semitones) to tame the Saw
        "filter_1_resonance": 0.2,
        
        # Ensure envelopes are active (if needed)
        "envelope_1_attack": 0.0,
        "envelope_1_decay": 1.5,
        "envelope_1_sustain": 0.0,
        "envelope_1_release": 0.5,
        
        # Try to enable Sample just in case (though likely empty)
        "sample_level": 1.0,
        "sample_destination": 0.0 # Filter 1
    }
    
    for name, value in updates.items():
        if name in plugin.parameters:
            plugin.parameters[name].raw_value = value
            print(f"Set {name} to {value}")

fix_piano_preset(plugin)

In [ ]:
def render_note(plugin, note_pitch, velocity=100, duration=4.0):
    sample_rate = 44100
    note_start = 0.5
    note_end = duration - 0.5
    
    # Flush again before render to ensure silence at start
    plugin.process([], 0.1, sample_rate, num_channels=2, reset=True)
    
    # MIDI Messages
    note_on = mido.Message("note_on", note=note_pitch, velocity=velocity).bytes()
    note_off = mido.Message("note_off", note=note_pitch, velocity=velocity).bytes()
    
    midi_messages = [
        (note_on, note_start),
        (note_off, note_end)
    ]
    
    # Render
    audio = plugin.process(
        midi_messages,
        duration,
        sample_rate,
        num_channels=2,
        buffer_size=2048,
        reset=True
    )
    return audio, sample_rate

# Generate Middle C
audio, sr = render_note(plugin, 60)

# Save and Play
output_path = os.path.join(OUTPUT_DIR, "piano_c4.wav")
sf.write(output_path, audio.T, sr)
print(f"Saved to {output_path}")

ipd.Audio(audio, rate=sr)